In [9]:
import requests
import xlsxwriter
import numpy as np
import pandas as pd
import pandas.io.sql
import pymssql   

import pymssql
datacompany = (
                {'host':'192.168.250.40' , 'dbname':'IGU_NEW','user':'sa','password':'B1admin'} ,
                )
                
datalist=[]
listcom = []
for company in datacompany:
    
    msg_sql=  """
                   select   iyear,
                            imonth,
                            iday,
                            cardname ,
                        sum( balance)balance
                    from (
                    SELECT year(a.docdate) iyear,
                            month(a.docdate) imonth,
                            right(convert(varchar,a.docdate,112),2) iday,
                            b.cardname ,
                        sum( a.DocTotal - a.PaidSys)balance
                    FROM OINV A
                    INNER JOIN OCRD B ON A.CARDCODE = B.CARDCODE 
                    INNER JOIN OCRG C ON B.GroupCode = C.GroupCode
                    where c.groupcode ='117'
                    group by year(a.docdate) ,
                            month(a.docdate) ,
                            right(convert(varchar,a.docdate,112),2) ,
                            b.cardname 
                    having sum( a.DocTotal - a.PaidSys) <>0

                    union all
                    SELECT year(a.docdate) iyear,
                            month(a.docdate) imonth,
                            right(convert(varchar,a.docdate,112),2) iday,
                            b.cardname ,
                        -1 * sum( a.DocTotal - a.PaidSys)balance
                    FROM orin A
                    INNER JOIN OCRD B ON A.CARDCODE = B.CARDCODE 
                    INNER JOIN OCRG C ON B.GroupCode = C.GroupCode
                    where c.groupcode ='117'
                    group by year(a.docdate) ,
                            month(a.docdate) ,
                            right(convert(varchar,a.docdate,112),2) ,
                            b.cardname 
                    having sum( a.DocTotal - a.PaidSys) <>0
                    )as a
                    group by iyear,
                            imonth,
                            iday,
                            cardname
                    order by   iyear,
                            imonth,
                            iday,
                            cardname
                """
   
    listcom.append(company["dbname"])
    conn = pymssql.connect(host=company["host"] , user=company["user"] , password=company["password"], database=company["dbname"] )
    cursor = conn.cursor()     
    data = pandas.io.sql.read_sql(msg_sql,conn)
    datalist.append(data)

df = pd.concat(datalist)  

df.pivot_table(index=['cardname'],columns=["iyear","imonth","iday"],aggfunc=np.sum,  values=["balance"],fill_value="0",margins=True).to_excel("/data/cabang.xlsx")
df.pivot_table(index=["iyear","imonth","iday"],columns=['cardname'],aggfunc=np.sum,  values=["balance"],fill_value="0",margins=True).to_excel("/data/cabang2.xlsx")
 
 
 
df.pivot_table(index=['cardname'],columns=["iyear","imonth"],aggfunc=np.sum,  values=["balance"],fill_value="0",margins=True).to_excel("/data/cabang3.xlsx")
 

 

/tmp/ipykernel_238617/3915939408.py:68: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pandas.io.sql.read_sql(msg_sql,conn)
